# Test Service

Intended to test the service.py evaluator.
Runs the service.py and a simple client.



# Setup

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
import PIL
from flatland.utils.rendertools import RenderTool
import imageio
import os

In [4]:
from IPython.display import clear_output
from IPython.core import display
import ipywidgets as ipw
display.display(display.HTML("<style>.container { width:95% !important; }</style>"))

In [5]:
import pickle

from flatland.envs.rail_env import RailEnv
from flatland.envs.rail_generators import sparse_rail_generator
from flatland.envs.schedule_generators import sparse_schedule_generator
from flatland.envs.malfunction_generators import malfunction_from_file, no_malfunction_generator
from flatland.envs.rail_generators import rail_from_file
from flatland.envs.schedule_generators import schedule_from_file
from flatland.core.env_observation_builder import DummyObservationBuilder

In [6]:
from flatland.envs.persistence import RailEnvPersister
from flatland.evaluators.client import FlatlandRemoteClient
import redis
import subprocess as sp
import shlex
import time
import pkg_resources as pr
import importlib_resources as ir
import sys

In [7]:
!pwd

/home/jeremy/projects/aicrowd/rl-trains/flatland5/notebooks


### Find the real path of the `env_data` package (should be copied by tox)

In [9]:
with ir.path("env_data.tests", "test_001.pkl") as oPath:
    sPath = oPath
print(type(sPath), sPath)

<class 'pathlib.PosixPath'> /home3/jeremy/projects/aicrowd/rl-trains/flatland5/env_data/tests/test_001.pkl


In [10]:
sDirRoot = "/" + "/".join(sPath.parts[1:-1] + ("service_test",""))
sDirRoot

'/home3/jeremy/projects/aicrowd/rl-trains/flatland5/env_data/tests/service_test/'

### Clear any old redis keys

In [11]:
oRedis = redis.Redis()

In [12]:
lKeys = oRedis.keys("flatland*")
lKeys

[b'flatland-rl::FLATLAND_RL_SERVICE_ID::response::65c5cdafbda515c05db3af5b2c7800ce']

In [13]:
for sKey in lKeys:
    print("Deleting:", sKey)
    oRedis.delete(sKey)

Deleting: b'flatland-rl::FLATLAND_RL_SERVICE_ID::response::65c5cdafbda515c05db3af5b2c7800ce'


## Service python command
### Kill any old service process

In [14]:
!ps -ef | grep -i python | grep -i flatland.evaluators.service | awk '{print $2}' | xargs kill

In [15]:
#sCmd = "python -m flatland.evaluators.service --test_folder ../env_data/tests/service_test --mergeDir ./tmp/merge --actionDir ./tmp/actions --pickle --missingOnly"
#sCmd = "python -m flatland.evaluators.service --test_folder ../env_data/tests/service_test --pickle" # --verbose"
sCmd = f"python -m flatland.evaluators.service --test_folder {sDirRoot} --pickle" # --verbose"
lsCmd = shlex.split(sCmd)
print(sCmd)
print(lsCmd)

python -m flatland.evaluators.service --test_folder /home3/jeremy/projects/aicrowd/rl-trains/flatland5/env_data/tests/service_test/ --pickle
['python', '-m', 'flatland.evaluators.service', '--test_folder', '/home3/jeremy/projects/aicrowd/rl-trains/flatland5/env_data/tests/service_test/', '--pickle']


In [16]:
#wOut = ipw.Output()
#wOut

In [17]:
oPipe = sp.Popen(lsCmd)

In [18]:
type(oPipe)

subprocess.Popen

In [19]:
oPipe.poll()

In [20]:
#oFRC = FlatlandRemoteClient(test_envs_root="../env_data/tests/service_test/", verbose=False, use_pickle=True)
oFRC = FlatlandRemoteClient(test_envs_root=sDirRoot, verbose=False, use_pickle=True)

In [21]:
#env, env_dict = RailEnvPersister.load_new("../env_data/tests/service_test/Test_0/Level_0.pkl") # env_file)
env, env_dict = RailEnvPersister.load_new(f"{sDirRoot}/Test_0/Level_0.pkl") # env_file)
ldActions = env_dict["actions"]

In [22]:
def expert_controller(obs, _env):
    return ldActions[_env._elapsed_steps]

def random_controller(obs, _env):
    dAct = {}
    for iAg in range(len(_env.agents)):
        dAct[iAg] = np.random.randint(0, 5)
    return dAct

In [23]:
oObsB = DummyObservationBuilder()

In [24]:
oObsB.get()

True

In [25]:
if True:
    episode = 0
    obs = True
    while obs:
        obs, info = oFRC.env_create(
            obs_builder_object=oObsB
        )
        if not obs:
            print("null observation!")
            """
            The remote env returns False as the first obs
            when it is done evaluating all the individual episodes
            """
            break
        print("Episode : {}".format(episode))
        episode += 1

        print(oFRC.env.dones['__all__'])

        while True:
            if episode < 3:
                action = expert_controller(obs, oFRC.env)
            else:
                action = random_controller(obs, oFRC.env)
                
            time_start = time.time()

            try:
                observation, all_rewards, done, info = oFRC.env_step(action)
                time_diff = time.time() - time_start
                #print("Step Time : ", time_diff)
                print(".", end="")
                if done['__all__']:
                    print("\nCurrent Episode : ", episode)
                    print("Episode Done")
                    print("Reward : ", sum(list(all_rewards.values())))
                    break
            except TimeoutException as err:
                print("Timeout: ", err)
                break

    print("Evaluation Complete...")
    print(oFRC.submit())

DEPRECATED - use FileMalfunctionGen instead of malfunction_from_file
DEPRECATED - RailEnv arg: malfunction_and_process_data - use malfunction_generator
Episode : 0
False
...........................................................................................................................................................................
Current Episode :  1
Episode Done
Reward :  10.0
DEPRECATED - use FileMalfunctionGen instead of malfunction_from_file
DEPRECATED - RailEnv arg: malfunction_and_process_data - use malfunction_generator
Episode : 1
False
...........................................................................................................................................................................
Current Episode :  2
Episode Done
Reward :  10.0
null observation!
Evaluation Complete...
## Client Performance Stats
	 - env_creation_wait_time	 => min: 0.0010077953338623047 || mean: 0.0071858565012613935 || max: 0.014672040939331055
	 - internal_env_reset_time	 =>

### Kill the evaluator process we started earlier

In [26]:
!ps -ef | grep -i python | grep -i flatland.evaluators.service | awk '{print $2}' | xargs kill